In [14]:
from classes import *
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
import geopandas as gpd
from shapely.geometry import Polygon as ShapelyPolygon
import json
import folium

from os import path

In [15]:
data1_path = "/Users/sebastiangmur/Projekte/Uzh/GEO877_Spatial_Algorithms/SpatialAlgorithms/data/Polygons_no_mountains.geojson"
data2_path = "/Users/sebastiangmur/Projekte/Uzh/GEO877_Spatial_Algorithms/SpatialAlgorithms/data/Polygons_with_mountains.geojson"
data3_path = "/Users/sebastiangmur/Projekte/Uzh/GEO877_Spatial_Algorithms/SpatialAlgorithms/data/swissnames_points_json_export.geojson"

json_files = [(data1_path, "Gemeinden_ohne_Berge"),
(data2_path, "Gemeinden_mit_Berge"),
(data3_path, "Swissnames_Points")]


## Read in Data (Marek)

Short description what happens and how

In [16]:
data = process_json_file(json_files)

## Preprocessing (Sebi)

Samesame

In [17]:
polygon_data = Polygon_Data(data['Gemeinden_mit_Berge'],data['Gemeinden_ohne_Berge'])

In [18]:
polygon_data.remove_unique_entries()

10 unique objects removed from municipalities_polygons:
Polygon Zwieselberg consisting of 6 points BFS_Nr=947
Polygon Wangenried consisting of 6 points BFS_Nr=993
Polygon Escholzmatt-Marbach consisting of 23 points BFS_Nr=1010
Polygon Lüterswil-Gächliwil consisting of 9 points BFS_Nr=2456
Polygon Bergün Filisur consisting of 36 points BFS_Nr=3544
Polygon Turgi consisting of 9 points BFS_Nr=4042
Polygon Novaggio consisting of 11 points BFS_Nr=5207
Polygon Törbel consisting of 11 points BFS_Nr=6296
Polygon Beurnevésin consisting of 7 points BFS_Nr=6773
Polygon Bonfol consisting of 9 points BFS_Nr=6775

16 unique objects removed from muns_only_vegetation_area_polygons:
Polygon Valsot consisting of 43 points BFS_Nr=3764
Polygon Ponte Capriasca consisting of 6 points BFS_Nr=5212
Polygon Rüeggisberg consisting of 17 points BFS_Nr=880
Polygon Zuoz consisting of 8 points BFS_Nr=3791
Polygon Samedan consisting of 14 points BFS_Nr=3786
Polygon Madulain consisting of 8 points BFS_Nr=3783
Polygon 

In [19]:
municipalities_mountain_area = polygon_data.apply_difference()

In [20]:
aeugst_am_albis = polygon_data.cleaned_mun_polys[0]
aeugst_am_albis_vf = polygon_data.cleaned_mun_only_vegetation_polys[0]


In [21]:
aeugst_am_albis_mountain_area = municipalities_mountain_area[0]

In [22]:
print(f"aeugst am albis fläche: {aeugst_am_albis.area()}")
print(f"nach diff: {test[0].area()}")

aeugst am albis fläche: 6985196.0
nach diff: 2981.276611328125
